In [60]:
import networkx as nx
import random

In [61]:
class CPM(nx.DiGraph):
    def __init__(self):
        super().__init__()
        self._dirty = True
        self._makespan = -1
        self._criticalPath = None

    def add_node(self, *args, **kwargs):
        self._dirty = True
        super().add_node(*args, **kwargs)

    def add_nodes_from(self, *args, **kwargs):
        self._dirty = True
        super().add_nodes_from(*args, **kwargs)

    def add_edge(self, *args):  # , **kwargs):
        self._dirty = True
        super().add_edge(*args)  # , **kwargs)

    def add_edges_from(self, *args, **kwargs):
        self._dirty = True
        super().add_edges_from(*args, **kwargs)

    def remove_node(self, *args, **kwargs):
        self._dirty = True
        super().remove_node(*args, **kwargs)

    def remove_nodes_from(self, *args, **kwargs):
        self._dirty = True
        super().remove_nodes_from(*args, **kwargs)

    def remove_edge(self, *args):  # , **kwargs):
        self._dirty = True
        super().remove_edge(*args)  # , **kwargs)

    def remove_edges_from(self, *args, **kwargs):
        self._dirty = True
        super().remove_edges_from(*args, **kwargs)

    def _forward(self):
        for n in nx.topological_sort(self):
            S = max([self.node[j]['C']
                     for j in self.predecessors(n)], default=0)
            self.add_node(n, S=S, C=S + self.node[n]['p'])

    def _backward(self):
        for n in nx.topological_sort(self, reverse=True):
            Cp = min([self.node[j]['Sp']
                      for j in self.successors(n)], default=self._makespan)
            self.add_node(n, Sp=Cp - self.node[n]['p'], Cp=Cp)

    def _computeCriticalPath(self):
        G = set()
        for n in self:
            if self.node[n]['C'] == self.node[n]['Cp']:
                G.add(n)
        self._criticalPath = self.subgraph(G)

    @property
    def makespan(self):
        if self._dirty:
            self._update()
        return self._makespan

    @property
    def criticalPath(self):
        if self._dirty:
            self._update()
        return self._criticalPath

    def _update(self):
        self._forward()
        self._makespan = max(nx.get_node_attributes(self, 'C').values())
        self._backward()
        self._computeCriticalPath()
        self._dirty = False

In [62]:
def generate_nodes(processes):
    graph_nodes = CPM()
    for process in processes:
        graph_nodes.add_node(process[0], p=process[1])
    return graph_nodes

In [63]:
def generate_random_solution(graph_nodes, no_processes, no_requests):
    # obtenemos lista en orden de los nodos
    nodes = graph_nodes.nodes()
    # creamos lista de listas vacias para organizar el orden de los procesos en comun
    node_list = [[] for _ in range(no_processes)]
    # separamos los nodos de procesos que pertenecen a cada pedido 
    adjacency_list = [nodes[i:i + no_processes] for i in range(0, len(nodes), no_processes)]
    # creamos los arcos para unir los nodos en el orden establecido por el proceso
    for lst in adjacency_list:
        for i in range(len(lst)-1):
            graph_nodes.add_edges_from([(lst[i], lst[i+1])])
    # agrupamos los nodos que pertenecen a la misma operacion para asignarlas al azar
    nodes = graph_nodes.nodes()
    for i in range(len(node_list)):
        for j in range(i, len(nodes), no_processes):
            node_list[i].append(nodes[j])
    # asignar el orden de uso de cada estacion aleatoriamente
    for common_processes in node_list:
        random.shuffle(common_processes)
        for i in range(len(common_processes)-1):
            graph_nodes.add_edges_from([(common_processes[i], common_processes[i+1])])
    # regresamos el grafo con precedencias
    return graph_nodes, node_list


In [142]:
requests = [(1,50),(2,3),(3,41),(4,94),(5,15),(6,13),(7,34),(8,29),(9,55),(10,73),(11,14),(12,91)]
requests2 = [(1,5),(2,3),(3,4),(4,9)]
no_processes = 4
x = generate_nodes(requests)
y, node_list = generate_random_solution(x, no_processes, int(len(requests)/no_processes))
print(x.edges())

[(1, 2), (1, 5), (2, 3), (2, 10), (3, 4), (3, 11), (4, 12), (5, 6), (5, 9), (6, 7), (6, 2), (7, 8), (9, 10), (10, 11), (11, 12), (11, 7), (12, 8)]


In [131]:
print(y.makespan)
print(y.edges())

363
[(1, 2), (1, 5), (2, 3), (2, 10), (3, 4), (3, 11), (4, 12), (5, 6), (6, 7), (7, 8), (9, 10), (9, 1), (10, 11), (10, 6), (11, 12), (11, 7), (12, 8)]


In [132]:
def perturbate_solution(graph, node_list, patch_size):
    while patch_size > 0:
        # seleccionamos un proceso aleatorio para modificar su orden
        selected_process = random.randint(0, len(node_list)-1)
        # obtenemos los nodos involucrados en el proceso seleccionado
        nodes = node_list[selected_process]
        #print(node_list)
        # creamos una lista de tuplas con esos nodos para eliminar sus conexiones
        tup_list = []
        for i in range(len(nodes)-1):
            tup_list.append((nodes[i], nodes[i+1]))
        # eliminamos las conexiones entre esos nodos
        graph.remove_edges_from(tup_list)
        # generamos un nuevo orden aleatorio
        random.shuffle(nodes)
        # creamos las conexiones nuevas en base al nuevo orden aleatorio
        for i in range(len(nodes)-1):
            graph.add_edges_from([(nodes[i], nodes[i+1])])
        patch_size -= 1
        node_list[selected_process] = nodes
    return graph, node_list

In [133]:
g, nodes = perturbate_solution(y, node_list, 2)
g.makespan

391

In [134]:
g, nodes = perturbate_solution(g, node_list, 2)
g.makespan

378

In [163]:
# CLEVER ALGORITHM: Bees Algorithm
# Author: Santiago E. Conant-Pablos, October 6, 2015

import numpy as np
import matplotlib.pyplot as plt

def objective_function(graph):
    """returns value of function to optimize"""
    return graph.makespan

def create_random_bee(graph, no_processes, no_orders):
    """create a random bee position"""
    return generate_random_solution(graph, no_processes, no_orders)

def create_neigh_bee(graph, node_list, patch_size):
    """create a bee inside a neighborhood"""
    return perturbate_solution(graph, node_list, patch_size)

def search_neigh(parent, neigh_size, patch_size, node_list):
    """search inside the neighborhood of a site"""
    neigh = []
    for i in range(neigh_size):
        bee = list(create_neigh_bee(parent, node_list, patch_size))
        if len(bee) < 3:
            bee.append(objective_function(bee[0]))
        else:
            bee[2] = objective_function(bee[0])
        neigh.append(bee)
    neigh.sort(key=lambda b: b[2])
    return neigh[0]

def create_scout_bees(requests, no_processes, no_orders, num_scouts):
    """creates scout bees for new sites"""
    return [create_random_bee(generate_nodes(requests), no_processes, no_orders) for i in range(num_scouts)]

def bees_algorithm(max_gens, requests, no_processes, num_bees, num_sites,
                   elite_sites, patch_size, patch_dec, e_bees, o_bees):
    """implements the Bees algorithm"""
    # el mejor encontrado
    best = None
    # creacion de una poblacion de soluciones aleatorias
    no_requests = int(len(requests)/no_processes)
    # pop[i] = (graph, solution, fitness); fitness se agrega en el for interno
    pop = [create_random_bee(generate_nodes(
        requests), no_processes, no_requests
    ) for i in range(num_bees)]
    
    # corre la optimizacion en terminos de generaciones
    for gen in range(max_gens):
        for bee in range(num_bees):
            pop[bee] = list(pop[bee])
            #print('---', pop[bee][0].edges())
            if len(pop[bee]) < 3:
                pop[bee].append(objective_function(pop[bee][0]))
            else:
                pop[bee][2] = objective_function(pop[bee][0])
        pop.sort(key = lambda b: b[2])
        #print('rr')
        if not best or pop[0][2] < best[2]:
            best = pop[0]
        next_gen = []
        for i, parent in enumerate(pop[:num_sites]):
            neigh_size = e_bees if i < elite_sites else o_bees
            next_gen.append(search_neigh(parent[0], neigh_size, patch_size, parent[1]))
        
        scouts = create_scout_bees(requests, no_processes, no_requests, num_bees - num_sites)
        pop = next_gen + scouts
        patch_size = patch_size * patch_dec
        print(" > it=%d, patch_size=%g, f=%g" % (gen+1,patch_size,best[2]))
    return best

# problem configuration
#problem_size = 2 # number of variables
#search_space = np.array([[-5, +5] for i in range(problem_size)],float) # domains
# algorithm configuration
max_gens = 100 # maximun number of generations
num_bees = 45
num_sites = 3
elite_sites = 1
patch_size = 3.0
patch_dec = 0.95 # decrease of patch size in each generation
e_bees = 7    # number of elite bees
o_bees = 2    # number of other bees
no_jobs = 4 # cantidad de operaciones para ensamblaje (incluye varias en paralelo)
requests = [(1,50),(2,3),(3,41),(4,94),(5,15),(6,13),(7,34),(8,29),(9,55),(10,73),(11,14),(12,91)]
best = bees_algorithm(max_gens, requests, no_jobs, num_bees, num_sites,
                      elite_sites, patch_size, patch_dec, e_bees, o_bees)
print("Done.\nBest Solution: f=%g, v=%s" % (best[2], best[1]))

 > it=1, patch_size=2.85, f=356
 > it=2, patch_size=2.7075, f=356
 > it=3, patch_size=2.57212, f=356
 > it=4, patch_size=2.44352, f=356
 > it=5, patch_size=2.32134, f=356
 > it=6, patch_size=2.20528, f=327
 > it=7, patch_size=2.09501, f=323
 > it=8, patch_size=1.99026, f=323
 > it=9, patch_size=1.89075, f=323
 > it=10, patch_size=1.79621, f=323
 > it=11, patch_size=1.7064, f=298
 > it=12, patch_size=1.62108, f=298
 > it=13, patch_size=1.54003, f=298
 > it=14, patch_size=1.46302, f=298
 > it=15, patch_size=1.38987, f=298
 > it=16, patch_size=1.32038, f=298
 > it=17, patch_size=1.25436, f=298
 > it=18, patch_size=1.19164, f=298
 > it=19, patch_size=1.13206, f=298
 > it=20, patch_size=1.07546, f=298
 > it=21, patch_size=1.02168, f=298
 > it=22, patch_size=0.970601, f=298
 > it=23, patch_size=0.922071, f=298
 > it=24, patch_size=0.875967, f=298
 > it=25, patch_size=0.832169, f=298
 > it=26, patch_size=0.79056, f=298
 > it=27, patch_size=0.751032, f=298
 > it=28, patch_size=0.713481, f=298
